In [ ]:
##Load Necessary libraries
import csv
import requests
from bs4 import BeautifulSoup
from dateutil import parser
from datetime import datetime

## open file & write Header
file = open('linkedin-joblist.csv', 'a', newline='', encoding='utf-8')
writer = csv.writer(file, delimiter='|')
writer.writerow(['Title', 'Company', 'Location','Link','job_time'])
                 
##Defining function to read the webpage & fetch the values for every job

def linkedin_scraper(webpage,i,strn, page_number):
    next_page = webpage +str(i)+strn+str(page_number)
    print(str(next_page))
    response = requests.get(str(next_page))
    soup = BeautifulSoup(response.content,'html.parser')
    jobs = soup.find_all('div', class_='base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card')
    jc=0
    if page_number < 1000:
        for job in jobs:
            job_title = job.find('h3', class_='base-search-card__title').text.strip()
            job_company = job.find('h4', class_='base-search-card__subtitle').text.strip()
            job_location = job.find('span', class_='job-search-card__location').text.strip()
            job_link = job.find('a', class_='base-card__full-link')['href']
            #job_time = job.find('time', attrs={'class':'job-search-card__listdate'})['datetime']
            if job.find('time', attrs={'class':'job-search-card__listdate'}) is None:
                dt = datetime.now()
            else:
                dt = job.find('time', attrs={'class':'job-search-card__listdate'})['datetime']
                print (dt)
            writer.writerow([job_title,job_company,job_location,job_link,dt])
        page_number = page_number + 25
        linkedin_scraper(webpage,i,strn, page_number)
    else:
        file.close()
        print('File closed')

for i in range(0,4):
    file = open('linkedin-joblist.csv', 'a', newline='', encoding='utf-8')
    writer = csv.writer(file, delimiter='|')
    print('i is',i)
    ## Calling the funcion with URL 
    linkedin_scraper('https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Analyst&location=Ireland&locationId=&geoId=104738515&f_TPR=r2592000&position=1&pageNum=',i,'&start=', 0)
                 
##Note Pagenumber needs to be updated to fetch next page dtails. Each function call works to fetcch the job in specified page number(pg) - This For loop is to execute thar logic
        